In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM, SpatialDropout1D
from keras.datasets import imdb

Using TensorFlow backend.


In [2]:
np.random.seed(42)

In [3]:
# Максимальное кол-во слов по частоте использования
max_features = 5000
# Максимальная длина рецензии в словах
maxlen = 80

In [4]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

## Предварительная обработка рецензий

In [5]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

## Создаем нейронную сеть

In [6]:
model = Sequential()
# Слой для векторного представления слов
model.add(Embedding(max_features, 32))
model.add(SpatialDropout1D(0.2))
# Слой долго-краткосрочной памяти
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) 
# Полносвязный слой
model.add(Dense(1, activation="sigmoid"))

In [7]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [8]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          160000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, None, 32)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
model.fit(X_train, y_train, batch_size=64, epochs=7,
         validation_data=(X_test, y_test), verbose=1)

Train on 25000 samples, validate on 25000 samples
Epoch 1/7
25000/25000 [==============================] - 74s 3ms/step - loss: 0.5121 - acc: 0.7454 - val_loss: 0.3882 - val_acc: 0.8290
Epoch 2/7
25000/25000 [==============================] - 72s 3ms/step - loss: 0.3746 - acc: 0.8404 - val_loss: 0.3750 - val_acc: 0.8346
Epoch 3/7
25000/25000 [==============================] - 74s 3ms/step - loss: 0.3422 - acc: 0.8586 - val_loss: 0.3796 - val_acc: 0.8306
Epoch 4/7
25000/25000 [==============================] - 74s 3ms/step - loss: 0.3145 - acc: 0.8723 - val_loss: 0.3667 - val_acc: 0.8376
Epoch 5/7
25000/25000 [==============================] - 71s 3ms/step - loss: 0.2994 - acc: 0.8778 - val_loss: 0.4007 - val_acc: 0.8286
Epoch 6/7
25000/25000 [==============================] - 72s 3ms/step - loss: 0.2851 - acc: 0.8859 - val_loss: 0.3944 - val_acc: 0.8329
Epoch 7/7
25000/25000 [==============================] - 79s 3ms/step - loss: 0.2644 - acc: 0.8942 - val_loss: 0.4010 - val_acc: 0.830

In [12]:
scores = model.evaluate(X_test, y_test,
                        batch_size=64)
print("Точность на тестовых данных: %.2f%%" % (scores[1] * 100))

25000/25000 [==============================] - 14s 557us/step
Точность на тестовых данных: 83.09%


In [13]:
model_json = model.to_json()
with open("imdb_model.json", "w") as f:
    f.write(model_json)

model.save_weights("imdb_model.h5")